<a href="https://colab.research.google.com/github/pa-shk/context_dependency_resolution/blob/main/T5_large_fine_tuning_2rca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Комментарии Аня

Получается, наш средний диалог

1. Порядка 5 реплик,
2. Около 300 символов,
3. В нем много пунктуации,
4. Тематически про увлечения, склонности и преференции (то же, что в исходных данных)

Тогда эти средние данные надо отделить от исходных, проверить на адекватность и сделать датасетом отправным.

По другим датасетам подумать, как их использовать для аугментации.

In [1]:
import json
import itertools

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from tqdm import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-q66f8d1l because the default path (/home/shkunov/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
import os

# 2RCa

## Data loading

In [3]:
# !wget https://github.com/anna-a-m/dataset-2rca/raw/refs/heads/main/data/2rca_checked_version.json -q

In [3]:
with open('2rca_checked_version.json') as f:
    data = json.load(f)

## Data samples

In [5]:
data[0]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_64c12a9b',
 'Phrase': 'Что читаешь? Мне нравится классика. Я тоже люблю пообщаться.',
 'Rewrite': 'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'}

In [6]:
data[1]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_5cd9dc06',
 'Phrase': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику люблю.',
 'Rewrite': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.'}

In [7]:
data[2]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_9d73f6c2',
 'Phrase': 'Ух ты, интересно.',
 'Rewrite': 'Ух ты, интересно, ты фиалки выращиваешь.'}

In [8]:
data[3]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.',
  'Ух ты, интересно, ты фиалки выращиваешь.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_2bfa24e4',
 'Phrase': 'Ты случайно не принц на белом коне? Я его очень жду.',
 'Rewrite': 'Ты случайно не принц на белом коне? Я принца на белом коне очень жду.'}

Saving sample of data for manual examination

In [9]:
history, phrase, rewrite = [], [], []

for sample in data:
    history.append(sample["History"])
    phrase.append(sample["Phrase"])
    rewrite.append(sample["Rewrite"])

In [10]:
df = pd.DataFrame({"history": history, "phrase": phrase, "rewrite": rewrite})

## HF datasets

In [11]:
from datasets import DatasetDict, Dataset

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_ds, test_ds = ds["train"], ds["test"]
test_ds = test_ds.train_test_split(test_size=0.5, shuffle=True, seed=42)
val_ds, test_ds = test_ds["train"], test_ds["train"]

ds["train"] = train_ds
ds["val"] = val_ds
ds["test"] = test_ds

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
})

In [13]:
ds['train'][0]

{'history': ['Привет.',
  'Привет.',
  'Как твоё настроение сегодня?',
  'Настроение отлично, скоро куплю машину, а твоё настроение как?',
  'Здорово, ты скоро купишь машину! У меня тоже настроение на позитиве! Я очень давно хочу в Бразилию, и, кажется, сегодня моя мечта сбудется! Вечером поедем с подругой в турфирму оплачивать наш тур в Бразилию. А у тебя есть мечта?',
  'Давно хочу завести лабрадора, прям с детства хочу собаку. Может, когда дочка подрастет заведу себе лабрадора.',
  'Моей собаке уже 5 лет, и я даже не представляю, как я могла жить без своей собаки раньше?! Я думаю, что у тебя всё получится и у вас скоро обязательно появится питомец! Ведь собаки такие милые! Что сегодня будешь готовить на ужин?'],
 'phrase': 'Сегодня будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.',
 'rewrite': 'Сегодня на ужин будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'}

# Model training

## Model loading

In [14]:
import random
import os
import numpy as np
import torch


def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(42)

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

MODEL_NAME = "ai-forever/ruT5-large"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [16]:
def preprocess_function(sample):
    inputs = [f"{context[-1]}</s>{phrase}</s>"
              for context, phrase in zip(sample["history"], sample["phrase"])]

    model_inputs = tokenizer(inputs, text_target=sample["rewrite"], max_length=128, truncation=True)
    return model_inputs

In [17]:
from transformers import DataCollatorForSeq2Seq


tokenized_gapping_ds = ds.map(preprocess_function, batched=True, batch_size=4)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=MODEL_NAME,
                                       pad_to_multiple_of=8)

Map:   0%|          | 0/4411 [00:00<?, ? examples/s]

/home/shkunov/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/551 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

In [18]:
tokenized_gapping_ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 551
    })
})

## Training

In [19]:
from peft import LoraConfig, get_peft_model
from peft.utils.peft_types import TaskType


config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q", "v"],
    task_type=TaskType.SEQ_2_SEQ_LM
)

peft_model = get_peft_model(model, config)

In [20]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [21]:
training_args = Seq2SeqTrainingArguments(
    report_to ="wandb",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_steps=10,
)


trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_gapping_ds["train"],
    eval_dataset=tokenized_gapping_ds["val"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


trainer.train()


/tmp/ipykernel_2259769/3390119170.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: pvlshkunov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.505700,0.754020
2,0.918900,0.599165
3,0.708700,0.517031
4,0.629100,0.492273
5,0.536000,0.462659
6,0.545700,0.449581
7,0.515300,0.444490
8,0.472200,0.446173
9,0.451800,0.444180
10,0.406800,0.441882


KeyboardInterrupt: 

In [ ]:
model = peft_model.merge_and_unload()

# Evaluation

In [79]:
import torch
from tqdm.notebook import tqdm

In [80]:
raw_test_results_new = []
model = model.to(DEVICE)

for i, encoding in tqdm(list(enumerate(tokenized_gapping_ds['test']['input_ids']))):
    input_ids=torch.tensor([encoding]).to(DEVICE)
    out = model.generate(inputs=input_ids,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=50,
                         num_beams=2,
                         early_stopping=True)
    out = tokenizer.decode(out[0][1:],
                           skip_special_tokens=True,
                           clean_up_tokenization_spaces=True)
    raw_test_results_new.append(out)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 20.75 MiB is free. Process 1266198 has 11.93 GiB memory in use. Including non-PyTorch memory, this process has 2.62 GiB memory in use. Of the allocated memory 2.52 GiB is allocated by PyTorch, and 1.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import pandas as pd

raw_test_results = pd.read_csv("ruT5-large_raw_test_results.csv", index_col=0)
raw_test_results

,model_out_raw,history,text,restored_text
0,"Я из Твери, а ты откуда?","['Привет.', 'Привет. Откуда ты?']","Я из Твери, а ты?","Я из Твери, а ты откуда?"
1,"Да, Питерский политех. Жаль, что не в месте уч...","['Привет.', 'Привет! Сколько тебе лет? Ты где-...","Да, именно Питерский. Жаль, что не в месте учи...","Да, именно Питерский политех. Жаль, что не вме..."
2,Мои дети - мои песики. Пока не нашла достойног...,"['Приветики! Как дела?', 'Привет! Всё хорошо, ...",Мои дети - мои песики. Пока не нашла достойног...,Мои дети - мои песики. Пока не нашла достойног...
3,Где живёшь? Живешь в городе где-то? Какие фрук...,"['Привет.', 'Привет.', 'Расскажи о себе.', 'Я ...",Где живёшь? В городе где-то? Какие любишь?,Где живёшь? В городе где-то? Какие фрукты любишь?
4,"Да, потепление это самое главное, с кем ты раб...","['Здравствуй. Как тебе погодка?', 'Привет, я д...","Да, это самое главное, с кем ты работаешь?","Да, это самое главное что весна придет через п..."
...,...,...,...,...
546,"Это очень круто, что хочешь жениться! На сколь...","['Привет.', 'Привет.', 'Ты откуда?', 'Я из Кие...",Это очень круто! На сколько большую семью ты х...,"Это очень круто, что ты хочешь сделать предлож..."
547,"Вау, опасность завораживает.","['Привет, меня зовут Игорь.', 'Привет, меня зо...","Вау, опасность завораживает.","Вау, ты прыгал с парашютом, опасность заворажи..."
548,В свободное время люблю играть в футбол. Любиш...,"['Привет! Как дела?', 'Привет, мои дела отличн...",В свободное время люблю играть в футбол. Любиш...,В свободное время люблю играть в футбол. Любиш...
549,И тебе спасибо большое!,"['Привет! Я Анна.', 'Привет, я Сергей.', 'Как ...",И тебе спасибо большое!,И тебе спасибо большое за общение!


In [97]:
# raw_test_results = pd.DataFrame(raw_test_results_new, columns=['model_out_raw'])

# raw_test_results['history'] = tokenized_gapping_ds['test']['history']
# raw_test_results['text'] = tokenized_gapping_ds['test']['phrase']
# raw_test_results['restored_text'] = tokenized_gapping_ds['test']['rewrite']

# raw_test_results.head(10)

In [98]:
# raw_test_results.to_csv(f"{MODEL_NAME.split('/')[-1]}_raw_test_results.csv")

In [99]:
import numpy as np
import sacrebleu
from rouge_metric import PyRouge

rouge = PyRouge(rouge_n=(4), skip_gap=4)


class RestorationFScore:

    def __init__(self, tokenizer, n_gram: int=2):
        self.n_gram = n_gram
        self.tokenizer = tokenizer

    def preprocess(self, sents):
        for sent in sents:
            sent_tokenize = self.tokenizer(sent)['input_ids']
            yield [tuple(sent_tokenize[i:i+self.n_gram]) for i, _ in enumerate(sent_tokenize)]

    def _itereval(self):
        for i, predictions in enumerate(self.predictions):
            restored_ngrams = set(predictions).difference(self.references[i])
            ngrams_in_ref = set(self.rewrites[i]).difference(self.references[i])
            interagree = ngrams_in_ref.intersection(restored_ngrams)
            if len(restored_ngrams):
                precision = len(interagree) / len(restored_ngrams)
            else:
                precision = 0.
            if len(ngrams_in_ref):
                recall = len(interagree) / len(ngrams_in_ref)
            else:
                recall = 0.
            if precision or recall:
                yield 2 * ((precision * recall) / (precision + recall))
            else:
                yield 0.

    def evaluate(self, predictions: list,
                 references: list, rewrites: list):
        self.predictions = [p for p in self.preprocess(predictions)]
        self.references = [p for p in self.preprocess(references)]
        self.rewrites = [p for p in self.preprocess(rewrites)]
        return np.mean(list(self._itereval()))

In [100]:
def callculate_metrics(row):
    global r
    r = row
    row["bleu_score"] = sacrebleu.corpus_bleu(row.model_out_raw, [row.text]).score
    rouge_scores = rouge.evaluate(row.model_out_raw,
                                  [[t] for t in row.text])
    for k in rouge_scores:
        row[k] = rouge_scores[k]['f']

    for n in range(1, 5):
        rf_score = RestorationFScore(tokenizer, n)
        row[f"rf_score_{n}"] = rf_score.evaluate(predictions=row.model_out_raw,
                                                 references=row.text,
                                                 rewrites=row.restored_text)
    return row

In [101]:
raw_test_results["type"] = "2rca"
raw_test_results = raw_test_results.groupby(by="type").agg(list)

In [102]:
raw_test_results.apply(callculate_metrics, axis=1).drop(columns=["model_out_raw", "history", "text", "restored_text"])

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,73.178692,0.734649,0.62598,0.539959,0.471456,0.734438,0.431407,0.337138,0.303119,0.282961
